In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [2]:
#set path
path = '/media/onno/Volume/GFS_T850/'

In [3]:
# coordinatez = [
# (62,54,0,8),#Germany
# (54,46,0,8), #Spain
# (46,38,0,8), #Ukraine
# (62,54,8,16),#Germany
# (54,46,8,16), #Spain
# (46,38,8,16), #Ukraine
# (62,54,16,24),#Germany
# (54,46,16,24), #Spain
# (46,38,16,24)] #Ukraine
# #loading GFS and ERA5 data for specific box 
lonz_0 = np.arange(-148,56.1,4,dtype=np.int64)
lonz_1 = np.arange(-144,60.1,4,dtype=np.int64)
latz_1 = np.arange(30,74.1,4,dtype=np.int64)
latz_0 = np.arange(34,78.1,4,dtype=np.int64)
lonz_0 = [int(i+360) if i<0 else int(i) for i in lonz_0]
lonz_1 = [int(i+360) if i<=0 else int(i) for i in lonz_1]

In [5]:
#Choose Forecast model: GFS or ERA5RF
fcst_modelz = ['GFS','ERA5RF']
for fcst_model in fcst_modelz:
    for i in range(len(latz_0)):
        for j in range(len(lonz_0)):
            lat_0=latz_0[i];lat_1=latz_1[i]
            lon_0=lonz_0[j];lon_1=lonz_1[j]
            #open anomaly files
            file_anom_p90 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p90_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
            file_anom_p10 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p10_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
            anom_p90 = xr.open_dataset(path+file_anom_p90).t.squeeze()
            anom_p10 = xr.open_dataset(path+file_anom_p10).t.squeeze()
            #Process data for warm extremes
            #Calculate dates of persitent extremes that are 4 consecutive days or longer
            anom_p90_spec = anom_p90 > 0
            iszero = np.concatenate(([0], np.equal(anom_p90_spec.values, True).view(np.int8), [0]))
            abs_diff = np.abs(np.diff(iszero))
            # Runs start and end where absdiff is 1.
            idxz = np.where(abs_diff == 1)[0].reshape(-1, 2)
            idx_bool = (idxz[:,1]-idxz[:,0])>=4
            hw_idxz = idxz[idx_bool,:]
            hw_dates = np.empty(hw_idxz.shape,dtype=object)
            for k,hw_idx in enumerate(hw_idxz):
                hw_date = [str(anom_p90.time[hw_idx[0]].values),
                           str(anom_p90.time[hw_idx[1]-1].values)]
                hw_dates[k]=hw_date
            np.save(path+'dates/pw_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1),hw_dates)
            np.savetxt(path+'dates/pw_lon_{}_{}_lat_{}_{}.txt'.format(lon_0,lon_1,lat_0,lat_1),hw_dates,fmt="%s")
            #Process data for cold extremes
            anom_p10_spec = anom_p10 < 0
            iszero = np.concatenate(([0], np.equal(anom_p10_spec.values, True).view(np.int8), [0]))
            abs_diff = np.abs(np.diff(iszero))
            # Runs start and end where absdiff is 1.
            idxz = np.where(abs_diff == 1)[0].reshape(-1, 2)
            idx_bool = (idxz[:,1]-idxz[:,0])>=4
            cw_idxz = idxz[idx_bool,:]
            cw_dates = np.empty(cw_idxz.shape,dtype=object)
            for k,cw_idx in enumerate(cw_idxz):
                cw_date = [str(anom_p10.time[cw_idx[0]].values),
                           str(anom_p10.time[cw_idx[1]-1].values)]
                cw_dates[k]=cw_date

            np.save(path+'dates/cw_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1),cw_dates)
            np.savetxt(path+'dates/cw_lon_{}_{}_lat_{}_{}.txt'.format(lon_0,lon_1,lat_0,lat_1),cw_dates,fmt="%s")
